In [ ]:
##########################################################################
#################           BASE DATA MANIPULATION         ###############
##########################################################################

In [ ]:
! gunzip -c UKBB.GWAS.EXOME.CAD.txt.gz | \
    awk 'NR==1 || ($7 > 0.01) && ($13 > 0.75) {print}' | \
    gzip  > UKBB.GWAS.EXOME.CAD.filter.txt.gz

In [ ]:
! gunzip -c UKBB.GWAS.EXOME.CAD.filter.txt.gz | awk \
    '!( ($4=="A" && $5=="T") || \
        ($4=="T" && $5=="A") || \
        ($4=="G" && $5=="C") || \
        ($4=="C" && $5=="G")) {print}' | \
    gzip > UKBB.GWAS.EXOME.CAD.filter.noambig.txt.gz

In [ ]:
! gunzip -c UKBB.GWAS.EXOME.CAD.filter.noambig.txt.gz | \
    awk '{ print $2}' | sort | uniq -d > duplicated.snp

In [ ]:
! gunzip -c UKBB.GWAS.EXOME.CAD.filter.noambig.txt.gz  | \
    grep -vf duplicated.snp |\
    gzip - > UKBB.GWAS.EXOME.CAD.filter.noambig.nodups.txt.gz

In [ ]:
##########################################################################
################          TARGET DATA MANIPULATION         ###############
##########################################################################

In [ ]:
! gunzip -c genotypes.vcf.gz | head -250000 | gzip > genotypes.HEAD.vcf.gz  #working with the first 250K lines

In [ ]:
! plink --vcf genotypes.HEAD.vcf.gz

In [ ]:
! mv plink.bed genotype.bed
! mv plink.fam genotype.fam
! mv plink.bim genotype.bim
! mv plink.log genotype.log

In [ ]:
! plink --bfile genotype --maf 0.01 --hwe 1e-6 --geno 0.01 --mind 0.35 --write-snplist \
      --make-just-fam --out genotype.QC 

In [ ]:
! plink --bfile genotype --list-duplicate-vars ids-only suppress-first

In [ ]:
! gunzip -c genotypes.HEAD.vcf.gz | grep "^[^##]" | cut -f3 | sort | uniq -d > plink.dupvar  #hack to create the dupvar file

In [ ]:
! plink --bfile genotype --exclude plink.dupvar  --make-bed --out genotype

In [ ]:
! plink --bfile genotype  --keep genotype.QC.fam --extract genotype.QC.snplist \
      --indep-pairwise 50 12 0.25 --out genotype.QC

In [ ]:
! plink --bfile genotype --extract genotype.QC.prune.in  --keep genotype.QC.fam \
      --het --out genotype.QC

In [ ]:
# Optional generate SNP Filtering files, i.e. SNPs that have mismatching alleles reported in the base and target data
# genotype.QC.adj.bim
# genotype.mismatch

In [ ]:
mv genotype.bim genotype.bim.bk
ln -s genotype.QC.adj.bim genotype.bim

In [ ]:
! plink --bfile genotype --extract genotype.QC.prune.in  --keep genotype.valid.sample \
      --check-sex --out genotype.QC  #NEED X CHROMOSOME

In [ ]:
! plink --bfile genotype --extract genotype.QC.prune.in --keep genotype.valid.sample \
      --rel-cutoff 0.125 --out genotype.QC

In [ ]:
! plink --bfile genotype --make-bed --keep genotype.QC.rel.id --out genotype.QC \
      --extract genotype.QC.snplist --exclude genotype.mismatch

In [ ]:
##########################################################################
###########################    POLYGENIC RISK SCORE        ###############
##########################################################################

In [ ]:
#OR = Odds Ratio is already in log. No need to transform...

#clumping
! plink --bfile genotype.QC \
    --clump-p1 1 --clump-r2 0.3 --clump-kb 50 \
    --clump genotype.QC.Transformed  \
    --clump-snp-field SNP --clump-field P --out genotype

In [ ]:
! awk 'NR!=1{print $3}' genotype.clumped >  genotype.valid.snp
! awk '{print $2,$10}' genotype.QC.Transformed > SNP.pvalue

In [ ]:
! echo "0.001 0 0.001" > range_list
! echo "0.05 0 0.05" >> range_list
! echo "0.1 0 0.1" >> range_list
! echo "0.2 0 0.2" >> range_list
! echo "0.3 0 0.3" >> range_list
! echo "0.4 0 0.4" >> range_list
! echo "0.5 0 0.5" >> range_list

In [ ]:
! plink --bfile genotype.QC \
      --score genotype.QC.Transformed 2 5 8 header \
      --q-score-range range_list SNP.pvalue \
      --extract genotype.valid.snp \
      --out genotype

In [ ]:
############################################################################
################   PCA ANALYSIS - POPULATION STRATIFICATION  ###############
############################################################################

In [ ]:
! plink --bfile genotype.QC --indep-pairwise 50 10 0.3 --out genotype

In [ ]:
! plink --bfile genotype.QC --extract genotype.prune.in --pca 20 --out genotype